***(4,8)-Matroids***

In this notebook, we demonstrate the techniques and procedures used to study matroids of rank 4 on 8 elements.

In [40]:
using Oscar
using Combinatorics
pm = Polymake



#pull simple 4-8 matroids from polymake databases
db = Polymake.Polydb.get_db();
collection = db["Matroids.Small"];
d4n8 = Polymake.Polydb.find(collection, Dict("RANK" => 4, "SIMPLE"=>true, "N_ELEMENTS"=>8));

In [4]:
cd("..")

In [42]:
currentDir = pwd() # make sure you are running from the main directory.
include(joinpath(currentDir, "src/fileHandling.jl"))
include(joinpath(currentDir, "src/matroid_realization.jl"))
include(joinpath(currentDir, "src/Jacobian_Criterion.jl"))
include(joinpath(currentDir, "src/reduceTSC.jl"))

matroid_to_reduced_TSC_min_basis (generic function with 1 method)

We begin by removing matroids whose realization spaces are known to be smooth. We apply the following lemma:

**Lemma:** Let $M$ be a $(4,8)$-matroid such that there exists an element $i\in [8]$ contained in exactly $0\leq k\leq 3$ rank $3$ flats of cardinality at least $4$. Then there exists a smooth morphism $Gr_{M}\to Gr_{M|[8]\setminus i}$.

**Remark:** Matroid duality induces an isomorphism $Gr_{M}\cong Gr_{M^*}$. In other words, $Gr_{M|[8]\setminus i}\cong Gr_{(M|[8]\setminus i)^*}$. By prior work, we know that all $(3,7)$ matroids have smooth thin Schubert cells, and therefore smooth realization spaces. Hence, we can restrict our search for singular realization spaces to those $(4,8)$ matroids that do not satisfy the hypotheses of the above lemma.

We begin by isolating connected $(4,8)$-matroids such that every element of $[8]$ is contained in at least $4$ rank $3$ flats of cardinality at least $4$. There are **92** such matroids. We save the data as a sting of "$0$"'s and "$*$"'s, where the entries correspond to elements of ${[8]\choose 4}$ in reverse lex order.

In [43]:
#counts number of planes an element of the ground set is in
function in_planes(n,L)

    Ln = [l for l in L if n in l]

    return(length(Ln))

end



d4C8 = subsets(collect(1:8), 4);
d4C8 = sort(d4C8, by =  x-> reverse(x));
#isolate simple connected matroids satisfying 4 lines property
planes_4 = []

for t in d4n8
    
   # println(t)

    Qt = Matroid(t)

    L = [h for h in hyperplanes(Qt) if length(h)>3]

    ns = [n for n in 1:8 if in_planes(n,L)>3]

    if (length(ns) == 8&&is_connected(Qt))
                push!(planes_4,to_revlex(Qt,d4C8))
    end
end

LoadError: BSONError: domain=15, code=13053, message=No suitable servers found (`serverSelectionTryOnce` set): [Failed to resolve 'db.polymake.org']

This data is precomputed in `d4n8/four_planes_connected_4_8.dat`.

In [34]:
planes_4_precomputed = vec(readlines(joinpath(currentDir, "d4n8/4planes_connected_4_8.dat")));

In [35]:
length(planes_4_precomputed)

92

In [9]:
Set(planes_4) == Set(planes_4_precomputed)

true

We now begin our analysis of realization spaces of $(4,8)$-matroids. Note that this computation relies on the existence of a circuit of cardinality $d+1$, called a *reference circuit*. Therefore, the first step of the following computation checks for the existence of such a circuit. Matroids without such a reference circuit will be studied via their thin Schubert cells. There is exactly one simple, connected, $(4,12)$-matroid without a reference circuit.

In [66]:
no_circ = []
for i in planes_4
    Qi = matroid_from_revlex_basis_encoding(i,4,8)
    Cs = rank_plus1_circuits(Qi)
    if length(Cs) == 0
        push!(no_circ,i)
    end
end

In [67]:
no_circ

Any[]

This data is precomputed in `d4n8/no_ref_circ_4_8.dat`.

In [12]:
Set(no_circ) == Set(vec(readlines(joinpath(currentDir, "d4n8/no_ref_circ_4_8.dat"))))

true

The data for the coordinate rings of matroid realization spaces consists of a polynomial ring, a multiplicative semigroup, and the defining ideal.

We sort the matroids into the following categories: 

1) Not realizable.

2) Defining ideal reduces to 0.

3) Defining ideal is univariate

**Note:** Different reference circuits lead to difference presenation of the coordinate ring of the realization space. Of course all presentations yield isomorphic realization spaces. However, we keep track of the reference circuit used for computing the coordinate ring using the following functions.

Then we save the data as a string of $*$'s and $0$'s, such that the first $8$ entries of the string tell us the reference circuit. That is, we have a "$*$" in the $i$th position if $i$ is an element of the referenc circuit, "$0$" otherwise. The remaining $70$ entries of the string correspond to the the elements of ${[8]\choose 4}$ listed in reverse lex order.


The precomputed data `d4n8/nonrealizable_4_8.dat` contains the $(4,8)$ matroids that are not $\mathbb{Q}$ realizable. There are $25$ such maroids.

In [73]:
nonrealizable = vec(readlines(joinpath(currentDir, "d4n8/nonrealizable_4_8.dat")))
length(nonrealizable)

25

Note that the first $8$ entries correspond to the reference circuit used in our intitial computation. Hence we perform our verifcation using entries 9 through 78

In [68]:
nonrealizable = vec(readlines(joinpath(currentDir, "d4n8/nonrealizable_4_8.dat")))

test_nonrealizable = []

for Qstr in not_realizable
    Q = matroid_from_revlex_basis_encoding(Qstr[9:78], 4, 8)
    As = rank_plus1_circuits(Q) 
    MR = new_matroid_realization_space(Q, As[1]; F=QQ, saturate=true)
    push!(test_nonrealizable, !MR.representable)
end

In [72]:
all(test_nonrealizable)

true

Now we check the realizable matroids in `d4n8/4planes_connected_4_8.dat` for the singularity properties of their realization spaces.

**Note:** One could check that these are smooth using the `singular_locus` function in `src/Jacobian_Criterion.jl`. To make this computation more transparent, we carry out the smoothness check explicitly.

The precomputed data `d4n8/zero_ideal_4_8.dat` contains the matroids in `d4n8/4planes_connected_4_8.dat` such that the vanishing ideal of their realization space reduces to the trivial ideal. There are **63** such matroids. These realization spaces of these matroids are of course smooth.

In [74]:
zero_ideal = vec(readlines(joinpath(currentDir, "d4n8/zero_ideal_4_8.dat")))
length(zero_ideal)

63

In [75]:
test_zero = []
for cir_Qstr in zero_ideal   
    Qstr = cir_Qstr[9:78]
    A = [i for i in 1:8 if string(cir_Qstr[i]) == "*"] 
    Q = matroid_from_revlex_basis_encoding(Qstr, 4, 8) 
    MR = new_matroid_realization_space(Q, A; F=QQ, saturate=true)
        
    MR = reduce_ideal_full(MR)
    I = MR.defining_ideal
    push!(test_zero, iszero(I))
end

In [63]:
all(test_zero)

true

The precomputed data `d4n8/univariate_ideal_4_8.dat` contains the matroids in `d4n8/4planes_connected_4_8.dat` for which there exists some reference circuit such that the vanishing ideal is univariate and therefore principal. There are **3** such matroids.

In [54]:
principal_univariate = vec(readlines(joinpath(currentDir,"d4n8/univariate_ideal_4_8.dat")))

3-element Vector{String}:
 "***00**00********0****0*****0**0*****************0****0***********0*******0***"
 "***00**00********0****0*****0**0*****************0****0********0************0*"
 "**0*00**000000*********0************0***0**0*************0***0******00********"

In [64]:
#verify univariate
test_univariate = []

for cir_Qstr in principal_univariate
    A = [i for i in 1:8 if string(cir_Qstr[i]) == "*"] 
    Qstr = cir_Qstr[9:78]
    Q = matroid_from_revlex_basis_encoding(Qstr, 4, 8) 
    MR = new_matroid_realization_space(Q, A; F=QQ, saturate=true)
    
    MR = reduce_ideal_full(MR)
    I = MR.defining_ideal
    length_vs = length(ideal_vars(gens(I)))
    push!(test_univariate, isone(length_vs))
end


In [56]:
all(test_univariate)

true

We now check if the realization spaces of those matroids contained in `d4n8/univariate_ideal_4_8.dat` are smooth.  By the Jacobian criterion, the affine scheme $\mathsf{Spec}(S^{-1}\mathbb{Q}[x_1,\ldots,x_n] / \langle f \rangle)$ is smooth if and only if the ideal

$$
J = \langle f, \frac{\partial f}{\partial x_1}, \ldots, \frac{\partial f}{\partial x_n}\rangle
$$

is the unit ideal in $S^{-1} \mathbb{Q}[x_1,\ldots,x_n]$. This is equivalent to the saturation $(J:S^{\infty})$ being the unit ideal in $\mathbb{Q}[x_1,\ldots,x_n]$. We perform this explicit check here. 

In [76]:

test_smooth_principal = []
for cir_Qstr in principal_univariate
    
    A = [i for i in 1:8 if string(cir_Qstr[i]) == "*"] 
    Qstr = cir_Qstr[9:78]
    Q = matroid_from_revlex_basis_encoding(Qstr, 4, 8) 
    MR = new_matroid_realization_space(Q, A; F=QQ, saturate=true)
        
    MR = reduce_ideal_full(MR)
    R = MR.ambient_ring
    x = gens(R)
    I = MR.defining_ideal
    Igens = gens(I)
    length(Igens) != 1 && error("not principal") 
    JM = jacobian_matrix(R, x, Igens)
    nr, nc = size(JM) 
    J = I + ideal(R, [JM[1,c] for c in 1:nc])
    Sing = stepwise_saturation(J, MR.inequations)
    #Sing = realization_space_2_singular_locus(MR)
    push!(test_smooth_principal, isone(Sing))
end

In [77]:
all(test_smooth_principal)

true

We now examine the remaining matroid. That is, the one without a reference circuit. We do so by studying its thin Schubert cell.

In [59]:
no_circs = vec(readlines(joinpath(currentDir,"d4n8/no_ref_circ_4_8.dat")))

1-element Vector{String}:
 "0********0****0*****0**0***00************00***0**0*****0****0********0"

In [60]:
M = matroid_from_revlex_basis_encoding(no_circs[1],4,8)
T = matroid_to_reduced_TSC_min_basis(M,QQ)

(QQMPolyRingElem[1], QQMPolyRingElem[x_{3, 1}, x_{2, 4}, x_{1, 3}, x_{4, 4}, x_{4, 2}, x_{4, 3}, x_{3, 4}], [1, 2, 3, 5])

As the defining ideal is the unit ideal, this matroid is not realizable over $\mathbb{Q}$.

Finally, we show that our sorting of the matroids in `d4n8/4_8_four_planes_connected.dat` was exhaustive.

In [61]:
all_matroids_with_circ = Set(union(principal_univariate,zero_ideal,not_realizable))
all_no_circs = union([cir_Qstr[9:78] for cir_Qstr in all_matroids_with_circ],no_circs)


Set(all_no_circs) == Set(planes_4_precomputed)

true